In [ ]:
from qat.lang.AQASM import *
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import PyLinalg
from qat.qpus import get_default_qpu
import numpy as np
import matplotlib.pyplot as plt
from math import gcd, modf
from labmath import multord  # pip install labmath
import contfrac # pip install contfrac
import string

# Algoritmo de Shor

Neste notebook, vamos implementar um algoritmo quântico para fatorar o número 15. Escreveremos um código de implementação do Algoritmo de Shor em `myQLM`, seguindo o material apresentado em sala.

Material de apoio:
- **Transformada de Fourier Quântica**: aula qft
- **Estimativa de Fase Quântica**: aula qpe
- **Algoritmo de Shor**: aula shor


## Revisão do Algoritmo de Shor
Lembrando da aula X, O Algoritmo de Shor pode ser utilizado para encontrar fatores de um número inteiro, $N$, sendo

$$N = pq$$

onde $p$ e $q$ são números primos. Isso é feito em quatro etapas principais, semelhantes à implementação da estimativa de fase quântica. Neste caso, usaremos dois registradores de qubits. O primeiro registro terá $n = \lceil \log_2 N^2 \rceil = \lceil 2 \log_2 N \rceil$ qubits e conterá os qubits de medição. O segundo registrador terá $m = \lceil \log_2 N \rceil$ qubits, e será o autoestado para a estimativa da fase quântica. Para o número $15$, podemos usar $m=n$, visto que a ordem é sempre uma potência de $2$.

1. Primeiro, começamos inicializando nossos qubits. Criamos uma superposição de todos os estados de base computacional $2^n$ nos $n$ qubits do primeiro registrador, aplicando uma porta Hadamard ($H$) em cada qubit começando no estado $\vert0\rangle^{\otimes n}$. Também inicializamos os qubits de destino $m = \lceil \log_2 N \rceil$  no estado $\vert1\rangle$. Aqui, o expoente $\otimes n$ significa que temos um produto tensorial dos estados de $n = m$ qubits.

2. Em segundo lugar, aplicamos o operador unitário $U_a|x\rangle = |ax \mod N\rangle $, que será usado para realizar a exponenciação modular no segundo registrador, controlando-o com cada um dos diferentes qubits do primeiro registrador. O esquema abaixo mostra a ordenação e as respectivas potências.
$$$$
<img src="shorimage.png" width=500 />

$$$$

3. Em terceiro lugar, aplicamos uma transformada de Fourier quântica inversa nos qubits do primeiro registrador.

4. Finalmente, medimos os primeiros $n$ qubits, colapsando para o estado $|l\rangle$ .

Depois que os resultados da medição forem determinados, precisaremos fazer um pós-processamento clássico adicional para determinar os fatores ou decidir executar o programa novamente.

5. Calculamos $\frac{l}{2^n}$ e tomamos o denominador como candidato a $r$. A expansão em uma fração continuada de $\frac{\ell}{2^n}$, onde $\ell$ é a saída do algoritmo, é dada por
$$\frac{\ell}{2^n}=\cfrac{1}{a_1 + \cfrac{1}{a_2 + \cfrac{1}{ \ddots + \cfrac{1}{a_k} }}}$$
Os sucessivos convergentes da fração continuada são $[a_1]$, $[a_1,a_2]$, $[a_1,a_2,a_3]$ e assim por diante onde
$$[a_1]=\cfrac{1}{a_1},\;\;[a_1,a_2]=\cfrac{1}{a_1 + \cfrac{1}{a_2 }},\;\;[a_1,a_2,a_3]=\cfrac{1}{a_1 + \cfrac{1}{a_2 + \cfrac{1}{ a_3}}}, \;\;\cdots$$
O último convergente é $[a_1,a_2,...,a_k]$, que é igual a $\frac{\ell}{2^m}$. No algoritmo de Shor, temos que escolher o convergente mais próximo de $\frac{\ell}{2^m}$ tal que o denominador seja menor do que $N$.

Por exemplo, os convergentes da expansão em fração continuada de $\ell/2^m$ para $\ell=85$ e $m=9$ são $[6]=\frac{1}{6}$, $[6,42]=\frac{42}{253}$, e por último $[6,42,2]=\frac{85}{512}=\ell/2^m$. Quando $N=21$, temos que selecionar o convergente $[6]=\frac{1}{6}$.

6. Testar se $r$ é período. Se $ a^r = 1 \mod N$, então $r$ é o período e é garantido que os fatores primos de $N$ serão obtidos através da relação

$$p,q = \text{mdc}(a^{r/2} \pm 1, N)$$






## Implementando o Algoritmo de Shor com o `myQLM`

Neste notebook, vamos implementar o Algoritmo de Shor para fatorar o número 15.


### 1. Inicializando os qubits

Inicializamos o registrador de medida como uma superposição uniforme através da operação $H^{\otimes n}|0\rangle^{\otimes n}$. O segundo registrador, onde será aplicada a exponenciação modular, é inicializado no estado $|1\rangle \equiv |00...001\rangle$.

Logo, o estado inicializado, $|\psi_0\rangle$, é dado por

$$|\psi_0\rangle = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}|i\rangle\otimes |1\rangle$$

## Exercício hands-on: Encontrar os fatores primos do número 15

Substitua os `FIXME`s pelas respostas corretas.

#### 1) Defina o número de qubits em cada registrador

Defina o inteiro, $N$, a ser fatorado e leia a descrição do Algoritmo de Shor para obter a resposta para o valor de cada registrador, sendo $n$ o número de qubits no primeiro registrador e $m$ o do segundo.

In [ ]:
# inteiro a ser fatorado
N = FIXME
# numero de qubits no primeiro registrador.
n = FIXME
# numero de qubits no segundo registrador.
m = FIXME
# numero total de qubits
num_qubits = FIXME

#### 2) Encontre possíveis valores de $a$

Esse valor será usado para calcular a exponenciação modular $a^r\mod15$. Note que ele deve ser coprimo com $N$, ou seja, $MDC(a,N)=1$, e deve ser menor que $N$.

In [ ]:
# Definir um valor para a

# Dicas: 
#   voce pode usar laços de repetição para definir diferentes valores possíveis
#   usar a função filter()
#   usar np.gcd() 

# Escreva seu código abaixo


#### 3) Código de inicialização dos registradores

    - Incialize o estado do primeiro registrador como uma superposição uniforme (use operações Hadamard).
    - O estado inicial do segundo registrador deve ser |000...001>.

In [ ]:
def initialize_qubits():

    init_circ = QRoutine()
    wires = init_circ.new_wires(num_qubits)

    # escreva seu codigo abaixo

    

    # fim do seu codigo

    return init_circ

In [ ]:
# visualizando o circuito de inicialização
# Você não precisa modificar nada nesta célula

circuit = Program()
qubits = circuit.qalloc(num_qubits)
circuit.apply(initialize_qubits(), qubits)
circuit = circuit.to_circ()
%qatdisplay circuit --svg

### 4 Exponenciação Modular Quântica

Nessa etapa, precisamos gerar os valores da exponenciação modular $a^{2^i} \mod 15$ no segundo registrador, e para cada inteiro $i$ será associado um qubit no primeiro registrador que irá controlar esta porta.

Para tal, precisamos implementar várias vezes o operador:

$U_a|y\rangle = |a y\mod 15\rangle$

Usaremos como exemplo, a implementação usada em https://arxiv.org/pdf/1202.6614.pdf para $a=7$, dado pelo circuito:

$$$$

<img src="M7.jpg" width=200 />

$$$$

Para cada $i$, basta aplicarmos este circuito $2^i$ vezes de forma controlada a partir do valor inicial 1.

O circuito acima pode ser controlado estendendo as portas $X$ para CNOTS com um qubit de controle e abrindo a porta SWAP em sua decomposição por CNOTs. Assim, cada CNOT pode ser controlado o estendendo para uma porta Toffoli (CCNOT) com o qubit de controle. Vide imagens abaixo:

$$$$

<img src="M7Aberto.jpg" width=400 />

$$$$

$$$$

<img src="CM7.jpg" width=400 />

$$$$

Para generalizar esta ideia com um $a$ diferente de 7, basta obter a representação do $U_a$ em CNOTS e SWAPS e convertê-lo para uma versão controlada. O material de referência mostra circuitos para outros valores de $a$.

#### Exercício

Faça o circuito de multiplicação modular para valores diferentes de $a$. Sugerimos fazer para $a=8$ e, caso sobre tempo, para $a=14$. Dica: Acompanhe o que foi feito para $a=7$ e, baseado na referência dada, reproduza de forma análoga para os outros valores de $a$.


In [ ]:
def controlled_modular_exponentiation(i):
    circ = QRoutine()
    wires = circ.new_wires(1 + n) #1 qubit de controle e 4 onde ocorrem a multiplicação

    # escreva seu codigo aqui


    # fim do seu codigo

    return circ
            

### 4) Criando o programa

Neste exercício, você terá que aplicar todos os blocos de operações do Algoritmo de Shor no circuito quântico.
O programa consiste na aplicação das três principais operações descritas acima:

1. Inicialização do circuito
2. Exponenciação modular
3. Transformada Quântica de Fourier (QFT) no orimeiro registrador

As medidas serão feitas e usadas no pós processamento a fim de se encontrar o período $r$.

Substitua as respostas corretas pelos 4 `FIXME`s que aparecem no código. 

In [ ]:
# criando o objeto shor com a classe Program
shor = Program()
# alocando qubits no programa shor
qr = shor.qalloc(FIXME)
# criando o circuito de inicialização e aplicando no circuito
init_circ = initialize_qubits()
shor.apply(FIXME, qr)

for i in range(4):
    modularExponentiation = controlled_modular_exponentiation(i)
    shor.apply(modularExponentiation,qr[3-i],qr[4],qr[5],qr[6],qr[7])

# Aplicando a QFT no primeiro registrador
# indique o numero de qubits e os qubits to registrador respectivamente.
QFT(FIXME)(FIXME)

# transformando de programa para circuito 
circuit = shor.to_circ()

# visualizando o circuito
# Obs: na interface web do jupyter é possível dar zoom na imagem e visualizar melhor o circuito
%qatdisplay circuit --svg

### Realizando uma única medida no primeiro registrador

In [ ]:
# submeta o circuito para job
# Repare que será realizada apenas uma medida no circuito 

job = circuit.to_job(nbshots = 1)
result = get_default_qpu().submit(job)

# Aqui imprimimos os estados medidos e suas respectivas probabilidades
# A lista states armazenará todos os estados medidos (ou apenas um unico estado se nbshots=1) para pos processamento
states = []
for sample in result:
     print("State %s: probability %s " % (sample.state, sample.probability))
     states.append(sample.state)

### Transformando o valor medido para decimal

Nesta etapa, vamos transformar o valor medido para decimal. O exercício que deverá fazer é transformar apenas o valor medido no primeiro registrador, que é o que será usado no pós processamento.

Corrija o `FIXME` para pegar apenas a medida no primeiro registrador e descartar a medida no segundo registrador.

In [ ]:
# O codigo abaixo converte os estados medidos no primeiro registrador em decimais
# Criamos uma lista string onde cada elemento corresponde a '0's ou '1's medidos no primeiro registrador 

string = [str(int(b)) for b in states[0]]

# Pegue apenas o estado do primeiro registrador (n qubits)
string_reg1 = FIXME

print("Um valor binário no 1o registrador: ", string_reg1)


string_reg1 = list(reversed(string_reg1))
decimal = int(''.join(string_reg1),2)

print("Valor decimal medido: ", decimal)

### Algoritmo de frações continuadas

O valor decimal medido é dado por $l$. Implemente o algoritmo de frações continuadas em $l/2^n$ para encontrar o candidato a $r$. Verifique se esse $r$ satisfaz $a^r \equiv 1\mod15$. Caso contrário, você terá que rodar o algoritmo novamente, realizando uma nova medida.

Dica: utilize o pacote do PyPi (https://pypi.org/project/ContFrac/) para implementar o algoritmo de frações continuadas, encontrar os convergentes e o valor candidado a ordem $r$. Você também pode implementar seu próprio código.

Corrija os `FIXMEs` para determinar o valor que será usado nas frações continuadas e, em seguida, implemente o código para encontrar o candidato a $r$.

In [ ]:
# corrija sabendo que value = l/2^n
value = FIXME/FIXME

# implemente o codigo para encontrar r aqui 


print("Candidato a r: ", r)
# fim do seu codigo
# Dica: se voce encontrar r ímpar, terá de rodar o algoritmo novamente

### Determinando os fatores primos de $N$

Nesta etapa final, você usará $r$ para encontrar os fatores primos de $N$. As ferramentas necessárias para resolver este problema estão descritas no começo deste notebook. Você também pode consultar o notebook da outra aula sobre o Algoritmo de Shor. 

In [ ]:
# Dica: voce pode usar a função no numpy, np.gcd() para facilitar o calculo do MDC.
# Escreva seu codigo aqui 

p = FIXME
q = FIXME
print(f"Os fatores primos encontrados são {p} e {q}.")
# fim do codigo

Se você não encontrou os fatores primos corretos, significa  que a medida que voce realizou não te forneceu a ordem $r$ correta. Você terá que rodar o algoritmo uma quantidade suficiente de vezes para encontrar o $r$ adequado. Você também pode aumentar o número de medidas `nbshots` e criar uma rotina que testa todos os valores medidos. Caso prefira, apenas rode o notebook mais algumas vezes.